In [ ]:
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torchvision.transforms
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import torch
import numpy as np
from keypoint_detection.utils.heatmap import gaussian_heatmap, generate_keypoints_heatmap, overlay_image_with_heatmap, get_keypoints_from_heatmap
from keypoint_detection.models.detector import KeypointDetector
from keypoint_detection.data.unlabeled_dataset import UnlabeledKeypointsDataset

In [ ]:
import wandb
from pathlib import Path
from skimage import io
import torchvision

In [ ]:
## Get Model checkpoint from wandb


checkpoint_reference = "tlips/icra-2022-workshop/model-2f4sjm76:latest"

# download checkpoint locally (if not already cached)
run = wandb.init(project="test-project", entity="tlips")
artifact = run.use_artifact(checkpoint_reference, type="model")
artifact_dir = artifact.download()

# 
checkpoint = torch.load(Path(artifact_dir) / "model.ckpt")
print(checkpoint["hyper_parameters"])
# load checkpoint
model = KeypointDetector.load_from_checkpoint(Path(artifact_dir) / "model.ckpt", backbone_type='Unet')

In [ ]:
IMAGE_DIR =  Path("..") / "test_dataset"
JSON_PATH =  Path("..")/  "test_dataset" / "dataset.json"
print(JSON_PATH)
dataset = UnlabeledKeypointsDataset(JSON_PATH,IMAGE_DIR, "corner_keypoints", "-1")
print(len(dataset))
dataloader = torch.utils.data.DataLoader(dataset, batch_size= 2, shuffle= True)

In [ ]:
def imshow(img):
    """
    plot Tensor as image
    images are kept in the [0,1] range, although in theory [-1,1] should be used to whiten..
    """
    np_img = img.numpy()
    # bring (C,W,H) to (W,H,C) dims
    img = np.transpose(np_img, (1,2,0))
    plt.imshow(img)
    plt.show()


In [ ]:
img = next(iter(dataloader))[0]

In [ ]:
transform  = torchvision.transforms.Resize((256,256))

In [ ]:
def show_results(show_extracted_keypoints = True, mode ="eval"):
    """
    show network outputs on the dataset.
    """
    plt.rcParams["figure.figsize"] = (20,3)
    pil_to_torch = torchvision.transforms.ToTensor()
    if mode == "eval":
        model.eval()
    else:
        model.train()
    for batch in iter(dataloader):
        with torch.no_grad():
            batch = transform(batch)
            channel = 0
            output = model(batch)[:,channel]
            if not show_extracted_keypoints:
                overlayed_heatmap = torch.stack(
                    [
                        pil_to_torch(overlay_image_with_heatmap(batch[i], torch.unsqueeze(output[i].cpu(), 0),0.6))
                        for i in range(batch.shape[0])
                    ]
                )
            else:
                n_keypoints = 4
                overlayed_heatmap = torch.stack(
                [
                    pil_to_torch(
                        overlay_image_with_heatmap(
                            batch[i],
                            torch.unsqueeze(
                                generate_keypoints_heatmap(
                                    batch.shape[-2:],

                                    get_keypoints_from_heatmap(output[i].cpu(), 1,n_keypoints),
                                    sigma=2,
                                    device = 'cpu'
                                ),
                                0,
                            ),
                            0.6
                        )
                    )
                    for i in range(batch.shape[0])
                ]
        )
        grid = torchvision.utils.make_grid(overlayed_heatmap, nrow=8)
        imshow(grid)


In [ ]:
show_results(True)